In [1]:
import sys
import os
import logging
import json
from dotenv import load_dotenv

# Change cwd to the DataEnrichment root folder
data_enrichment_root_path = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
os.chdir(data_enrichment_root_path)

if data_enrichment_root_path not in sys.path:
    sys.path.append(data_enrichment_root_path)

load_dotenv()

import db
from worker import process
from utils import download
from utils.get_video_description import get_video_description
from AudioProcessor.transcriber import transcribe_audio
from AudioProcessor.sentiment import SentimentAnalyzer
import AlignmentCalculator
import VideoProcessor
import AudioProcessor

/Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
db_conn = db.get_connection()
video_id = "7520882134135868695"

INFO - [EnrichmentWorker] - Successfully Connected to Database


In [3]:
video_path = os.path.abspath("./tmp/video/7520882134135868695.mp4")
audio_path = os.path.abspath("./tmp/video/7520882134135868695.wav")

transcript = """
Călin Georgescu este născut în ziua de 26 sub arhetipul împăratului.
 Nu este întâmplător că el vorbește despre ordine și adevăr.
 Este un destin care cere viziune, putere și disciplină.
 Dar în umbră apare conflictul cu autoritățile și lupta între adevăr și control.
 Tu ce crezi? Călin Georgescu este un lider sau un luptător cu sistemul?
 Dacă vrei să afli ziua ta de naștere, intră pe site-ul meu www.ralucaleahu.ro unde găsești karma zilei tale de naștere.
"""

sentiment_results = {
    'positive': 0.4230826497077942,
    'neutral': 0.0,
    'negative': 0.5769173502922058
 }

description = """
 🎯 Călin Georgescu s-a născut sub arhetipul Împăratului. O zi care cere ordine, autoritate, viziune. Dar și o karmă grea: să nu abuzezi de putere și să-ți supui ego-ul Adevărului. 👁\u200d🗨 În umbră, apare lupta cu autoritățile și conflictul dintre vocea interioară și sistem. ❓Tu ce vezi în el? Un lider? Sau un luptător cu sistemul? 👇 Lasă-mi părerea ta. 🔗 Și intră pe  https://www.ralucaleahu.ro/  să descoperi karma zilei tale de naștere. #ralucaleahu #calingeorgescupresedinte #suveranitate #foryiurpage #tiktokromania🇷🇴 #tarot
 """

In [4]:
analysis_result_json = VideoProcessor.process(video_path, transcript, sentiment_results, description)
analysis_result_json

INFO - [EnrichmentWorker] - Starting video processing for /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/video/7520882134135868695.mp4
INFO - [EnrichmentWorker] - AFC is enabled with max remote calls: 10.
INFO - [EnrichmentWorker] - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/projects/zeruel-ney/locations/global/publishers/google/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
INFO - [EnrichmentWorker] - AFC remote call 1 is done.
INFO - [EnrichmentWorker] - VIDEO Analysis result {'summary': "The video discusses Călin Georgescu's birth under the archetype of 'The Emperor,' suggesting it signifies a destiny requiring vision, power, and discipline, but also carries a karma of not abusing power and submitting ego to truth. It alleges that in his 'shadow,' conflict with authorities and a struggle between truth and control emerge. The speaker poses a question to the audience about whether Georgescu is a leader or a fighter against the syst

{'summary': "The video discusses Călin Georgescu's birth under the archetype of 'The Emperor,' suggesting it signifies a destiny requiring vision, power, and discipline, but also carries a karma of not abusing power and submitting ego to truth. It alleges that in his 'shadow,' conflict with authorities and a struggle between truth and control emerge. The speaker poses a question to the audience about whether Georgescu is a leader or a fighter against the system and invites them to visit a website to discover their own birth day karma.",
 'identified_subjects': [{'subject': 'calin georgescu', 'stance': 0.6},
  {'subject': 'fighter against the system', 'stance': 0.7}],
 'overall_alignment': -0.9}

In [6]:
identified_subjects = analysis_result_json["identified_subjects"]
llm_overall_alignment = analysis_result_json["overall_alignment"]
final_alignment, deterministic_alignment, alignment_conflict = AlignmentCalculator.calculate(identified_subjects, llm_overall_alignment, alpha=0.25)
print(f"llm_overall_alignment: {llm_overall_alignment} \nfinal_alignment: {final_alignment} \ndeterministic_alignment: {deterministic_alignment} \nalignment_conflict: {alignment_conflict}")

llm_overall_alignment: -0.9 
final_alignment: -0.8333333333333334 
deterministic_alignment: -0.6333333333333333 
alignment_conflict: 0.2666666666666667
